In [ ]:
# export
from fastai.datasets import URLs, untar_data
from pathlib import Path
import torch, re, PIL, os, mimetypes, csv
import matplotlib.pyplot as plt
from collections import OrderedDict
from typing import *
import pandas as pd, numpy as np
from enum import Enum
from torch import tensor

## Data block API from config class

### Core helper functions

In [ ]:
def noop(x): return x

In [ ]:
# export
def listify(o):
    if o is None: return []
    if isinstance(o, list): return o
    if isinstance(o, str): return [o]
    if isinstance(o, Iterable): return list(o)
    return [o]

In [ ]:
# export
def compose(x, funcs, *args, order_key='_order', **kwargs):
    key = lambda o: getattr(o, order_key, 0)
    for f in sorted(listify(funcs), key=key): x = f(x, **kwargs)
    return x

In [ ]:
# export
def uniqueify(x, sort=False):
    res = list(OrderedDict.fromkeys(x).keys())
    if sort: res.sort()
    return res

In [ ]:
# export
def setify(o): return o if isinstance(o,set) else set(listify(o))

def _get_files(p, fs, extensions=None):
    p = Path(p)
    res = [p/f for f in fs if not f.startswith('.')
           and ((not extensions) or f'.{f.split(".")[-1].lower()}' in extensions)]
    return res

def get_files(path, extensions=None, recurse=False, include=None):
    path = Path(path)
    extensions = setify(extensions)
    extensions = {e.lower() for e in extensions}
    if recurse:
        res = []
        for i,(p,d,f) in enumerate(os.walk(path)): # returns (dirpath, dirnames, filenames)
            if include is not None and i==0: d[:] = [o for o in d if o in include]
            else:                            d[:] = [o for o in d if not o.startswith('.')]
            res += _get_files(p, f, extensions)
        return res
    else:
        f = [o.name for o in os.scandir(path) if o.is_file()]
        return _get_files(path, f, extensions)

### Processor and ItemGetter

In [ ]:
# export
class Processor():
    def __call__(self, items):  return items
    def process1(self, item):   return item
    def deprocess1(self, item): return item
    def deprocess(self, items): return items

In [ ]:
# export
class ItemGetter():
    _default_proc = None
    def __init__(self, procs=None): 
        self.procs = [p() for p in listify(self._default_proc)] if procs is None else procs
    def __call__(self, items): return compose(items, self.procs)
    
    def get(self, o): return o  #How to get the actual item from o (example: fn -> Image, idxs -> 1hot encoded tensor)
    def raw(self, o): return o  #Undoes get when needed for representation (1hot encoded tensor -> idxs)
    
    def show(self, x, ax):                       raise NotImplementedError #How to show one element
    def show_xys(self, xs, ys, y_get, **kwargs): raise NotImplementedError #How to show a bunch of xs and ys

### Data block API core

One ItemList to contain them all

In [ ]:
# export
class ItemList():
    def __init__(self, items, item_get=None):
        self.item_get = ItemGetter() if item_get is None else item_get
        self.items = self.item_get(listify(items))
    def _get(self, i): return self.item_get.get(i)
    def __getitem__(self, idx):
        try: return self._get(self.items[idx])
        except TypeError:
            if isinstance(idx[0],bool):
                assert len(idx)==len(self) # bool mask
                return [self._get(o) for m,o in zip(idx,self.items) if m]
            return [self._get(self.items[i]) for i in idx]
    def __len__(self): return len(self.items)
    def __iter__(self): return iter(self.items)
    def __setitem__(self, i, o): self.items[i] = o
    def __delitem__(self, i): del(self.items[i])
    def __repr__(self):
        res = f'{self.__class__.__name__} ({len(self)} items)\n{self.items[:10]}'
        if len(self)>10: res = res[:-1]+ '...]'
        return res
    
    def deproc(self, item):
        item = self.item_get.raw(item)
        for proc in reversed(listify(self.item_get.procs)):
            item = proc.deproc1(item)
        return item
    
    def obj(self, idx):
        isint = isinstance(idx, int) or (isinstance(idx,torch.LongTensor) and not idx.ndim)
        item = self[idx]
        return self.deproc(item) if isint else [self.deprocess(o) for o in item]

LabeledData is just a convenience wrapper other xs and ys

In [ ]:
# export
TfmY = Enum('TfmY', 'No Mask Image Point Bbox')

In [ ]:
class AddXContext():
    def __init__(self, il,x): self.il,self.x = il,x 
    def __enter__(self, *args):
        self.il.item_get._x = self.x
        return self.il
    def __exit__(self, *args): self.il.item_get._x = None

In [ ]:
# export
class LabeledData():
    def __init__(self, x, y, tfms=None, tfm_y=TfmY.No):  
        self.x,self.y,self.tfms,self.tfm_y = x,y,tfms,tfm_y
    def __repr__(self):        return f'{self.__class__.__name__}\nx: {self.x}\ny: {self.y}\n'
    def __getitem__(self,idx):
        #TODO? Make below works if idx is a mask/array/list
        x = self.x[idx]
        with AddXContext(self.y, x) as yil: y = yil[idx] 
        return compose((x,y), self.tfms, tfm_y=self.tfm_y)
    def __len__(self):         return len(self.x)
    
    def deproc(self, o):
        x = self.x.deproc(o[0])
        with AddXContext(self.y, x) as yil: y = yil.deproc(o[1])
        return (x,y)

Dataloader and DataBunch

In [ ]:
# export
from torch.utils.data.dataloader import DataLoader
def get_dl(ds, bs, shuffle=False, drop_last=False, **kwargs):
    return DataLoader(ds, batch_size=bs, shuffle=shuffle, drop_last=drop_last, **kwargs)

In [ ]:
# export
class DataBunch():
    def __init__(self, train_dl, valid_dl):
        self.train_dl,self.valid_dl = train_dl,valid_dl
        
    @property
    def train_ds(self): return self.train_dl.dataset
    @property
    def valid_ds(self): return self.valid_dl.dataset
    
    def show_batch(self, is_valid=False, items=9, **kwargs):
        xb,yb = next(iter(self.valid_dl if is_valid else self.train_dl))
        xs,ys = [],[]
        for i in range(items):
            x,y = self.train_ds.deproc((xb[i], yb[i]))
            xs.append(x); ys.append(y)
        self.train_ds.x.item_get.show_xys(xs, ys, self.train_ds.y.item_get, **kwargs)

Main class to represent any kind of data. User provides the `get_x_cls` and `get_y_cls` then the four functions. At init everything is constructed with optionals transforms or custom instances of `get_x`/`get_y`.

In [ ]:
# export
class DataBlock():
    get_x_cls = ItemGetter
    get_y_cls = ItemGetter
    def get_source(self):         raise NotImplementedError
    #Return the source of your data (path, dataframe...) and optionally download it
    def get_items(self, source):  raise NotImplementedError
    #Use source to return the list of all items
    def split(self, items):       raise NotImplementedError
    #Explain how so split the items. Return two list of integers or two boolean masks
    def label(self, items):       raise NotImplementedError
    #Explain how to label your items. Return a list of labels
        
    def __init__(self, tfms=None, tfm_y=TfmY.No, get_x=None, get_y=None):
        self.source = self.get_source()
        items = ItemList(self.get_items(self.source)) #Just for fancy indexing
        split_idx = self.split(items)
        labels = ItemList(self.label(items))
        if get_x is None: get_x = self.get_x_cls()
        if get_y is None: get_y = self.get_y_cls()
        x_train,x_valid = map(lambda o: ItemList(items[o],  item_get=get_x), split_idx)
        y_train,y_valid = map(lambda o: ItemList(labels[o], item_get=get_y), split_idx)
        self.train = LabeledData(x_train, y_train, tfms=tfms, tfm_y=tfm_y)
        self.valid = LabeledData(x_valid, y_valid, tfms=tfms, tfm_y=tfm_y)
    
    def databunch(self, bs=64, **kwargs):
        dls = [get_dl(ds, bs, shuffle=s, drop_last=s, **kwargs) for (ds, s) in zip([self.train, self.valid], [True,False])]
        return DataBunch(*dls)

### First ItemGetters

How to get images

In [ ]:
# export
class ImageGetter(ItemGetter):
    def __init__(self, procs=None, cmap=None, alpha=1.): 
        super().__init__(procs)
        self.cmap,self.alpha = cmap,alpha
    def get(self, fn): return PIL.Image.open(fn)
    def show(self, x, ax):
        ax.imshow(x[0] if x.shape[0] == 1 else x.permute(1,2,0), cmap=self.cmap, alpha=self.alpha)
        ax.axis('off')
    def show_xys(self, xs, ys, y_get, cols=3, figsize=None):
        rows = (len(xs) // cols if len(xs)%cols == 0 else len(xs)//cols+1) 
        if figsize is None: figsize = (cols*3, rows*3)
        fig,axs = plt.subplots(rows, cols, figsize=figsize)
        for x,y,ax in zip(xs, ys, axs.flatten()):
            self.show(x, ax)
            y_get.show(y, ax)

How to get categories

In [ ]:
# export
class CategoryProcessor(Processor):
    def __init__(self): self.vocab=None
    
    def __call__(self, items):
        #The vocab is defined on the first use.
        if self.vocab is None:
            self.vocab = uniqueify(items)
            self.otoi  = {v:k for k,v in enumerate(self.vocab)}
        return [self.proc1(o) for o in items]
    def proc1(self, item):  return self.otoi[item]
    
    def deprocess(self, idxs):
        assert self.vocab is not None
        return [self.deproc1(idx) for idx in idxs]
    def deproc1(self, idx): return self.vocab[idx]

In [ ]:
# export
class CategoryGetter(ItemGetter):
    _default_proc = CategoryProcessor
    def show(self, x, ax): ax.set_title(x)

### Helper functions

To get image files

In [ ]:
# export
image_extensions = set(k for k,v in mimetypes.types_map.items() if v.startswith('image/'))

In [ ]:
# export
def get_image_files(path, include=None):
    return get_files(path, extensions=image_extensions, recurse=True, include=include)

To split

In [ ]:
# export
def random_splitter(items, valid_pct=0.2, seed=None): 
    if seed is not None: torch.manual_seed(seed)
    rand_idx = torch.randperm(len(items))
    cut = int(valid_pct * len(items))
    return rand_idx[:cut],rand_idx[cut:] 

In [ ]:
# export
def _grandparent_mask(items, name):
    return [(o.parent.parent.name if isinstance(o, Path) else o.split(os.path.sep)[-2]) == name for o in items]

def grandparent_splitter(items, train_name='train', valid_name='valid'):
    return _grandparent_mask(items, train_name),_grandparent_mask(items, valid_name)

To label

In [ ]:
# export
def parent_labeller(items):
    return [o.parent.name if isinstance(o, Path) else o.split(os.path.sep)[-1] for o in items]

In [ ]:
# export
def func_labeller(items, func):
    return [func(o) for o in items]

In [ ]:
# export
def re_labeller(items, pat):
    pat = re.compile(pat)
    def _inner(o):
        res = pat.search(str(o))
        assert res,f'Failed to find "{pat}" in "{s}"'
        return res.group(1)
    return func_labeller(items, _inner)

### First data:

In [ ]:
class PetsData(DataBlock):
    get_x_cls = ImageGetter
    get_y_cls = CategoryGetter
    
    def get_source(self):        return untar_data(URLs.PETS)
    def get_items(self, source): return get_image_files(source/"images")
    def split(self, items):      return random_splitter(items)
    def label(self, items):      return re_labeller(items, pat = r'/([^/]+)_\d+.jpg$')

In [ ]:
data = PetsData()

In [ ]:
img,cls = data.train[0]

In [ ]:
img

In [ ]:
img.size

In [ ]:
cls

In [ ]:
data.train.y.obj(0)

## Transforms

In [ ]:
# export
class Transform(): 
    _order=0
    _tfm_y_func={TfmY.Image: 'apply_img',   TfmY.Mask: 'apply_mask', TfmY.No: 'noop',
                 TfmY.Point: 'apply_point', TfmY.Bbox: 'apply_bbox'}
    
    def apply(self, x):       return x
    def apply_img(self, y):   return self.apply(y)
    def apply_mask(self, y):  return self.apply_img(y)
    def apply_point(self, y): return y
    def apply_bbox(self, y):  return self.apply_point(y)
    
    def randomize(self): pass
    
    def __call__(self, o, tfm_y=TfmY.No):
        (x,y) = o
        self.randomize() #Ensures we have the same state for x and y
        self.x = x #Saves the x in case it's needed in the apply for y (x.size for apply_point for instance)
        return self.apply(x),getattr(self, self._tfm_y_func[tfm_y], noop)(y)

In [ ]:
class DecodeImg(Transform):
    def __init__(self, mode_x='RGB', mode_y=None):
        self.mode_x,self.mode_y = mode_x,mode_y
        
    def apply(self, x):       return x.convert(self.mode_x)
    def apply_image(self, y): return y.convert(self.mode_x if self.mode_y is None else self.mode_y)
    def apply_mask(self, y):  return y.convert('L' if self.mode_y is None else self.mode_y)

In [ ]:
class ResizeFixed(Transform):
    _order=10
    def __init__(self, size, mode_x=PIL.Image.BILINEAR, mode_y=None):
        if isinstance(size,int): size=(size,size)
        size = (size[1],size[0]) #PIL takes size in the otherway round
        self.size,self.mode_x,self.mode_y = size,mode_x,mode_y
        
    def apply(self, x):       return x.resize(self.size, self.mode_x)
    def apply_image(self, y): return y.resize(self.size, self.mode_x if self.mode_y is None else self.mode_y)
    def apply_mask(self, y):  return y.resize(self.size, PIL.Image.NEAREST if self.mode_y is None else self.mode_y)
    def apply_point(self, y):
        rsz = tensor([self.x.size[1]/self.size[0], self.x.size[0]/self.size[1]], device=y.device)
        return y / rsz
    def apply_bbox(self, y): return self.apply_point(y[0]),y[1]

In [ ]:
class ToByteTensor(Transform):
    _order=20
    
    def apply(self, x):
        res = torch.ByteTensor(torch.ByteStorage.from_buffer(x.tobytes()))
        w,h = x.size
        return res.view(h,w,-1).permute(2,0,1)

In [ ]:
class ToFloatTensor(Transform):
    _order=20
    def __init__(self, div_x=255., div_y=None):
        self.div_x,self.div_y = div_x,div_y
    def apply(self, x):      return x.float().div_(self.div_x)
    def apply_mask(self, x): 
        return x.long() if self.div_y is None else x.long().div_(self.div_y)

In [ ]:
tfms = [DecodeImg(), ResizeFixed(128), ToByteTensor(), ToFloatTensor()]

In [ ]:
data = PetsData(tfms=tfms)

In [ ]:
datab = data.databunch()

In [ ]:
datab.show_batch()

In [ ]:
x,y = next(iter(datab.train_dl))

In [ ]:
x.shape

## Try different data

### MNIST

In [ ]:
class MnistData(DataBlock):
    get_x_cls = ImageGetter
    get_y_cls = CategoryGetter
    
    def get_source(self):        return untar_data(URLs.MNIST)
    def get_items(self, source): return get_image_files(source)
    def split(self, items):      return grandparent_splitter(items, train_name='training', valid_name='testing')
    def label(self, items):      return parent_labeller(items)

In [ ]:
data = MnistData(tfms=[ToByteTensor(), ToFloatTensor()]).databunch()

In [ ]:
data.show_batch()

In [ ]:
data.train_ds.x.item_get.cmap='gray'

In [ ]:
data.show_batch()

### Planet

In [ ]:
path = untar_data(URLs.PLANET_SAMPLE)

In [ ]:
path.ls()

In [ ]:
df = pd.read_csv(path/'labels.csv')

In [ ]:
df.head()

In [ ]:
# export
def onehot(x, c):
    res = torch.zeros(c)
    res[x] = 1.
    return res

In [ ]:
# export
class MultiCategoryProcessor(CategoryProcessor):
    def __call__(self, items):
        #The vocab is defined on the first use.
        if self.vocab is None:
            vocab = set()
            for c in items: vocab = vocab.union(set(c))
            self.vocab = list(vocab)
            self.vocab.sort()
            self.otoi  = {v:k for k,v in enumerate(self.vocab)}
        return [self.proc1(o) for o in items]
    def proc1(self, item):  return [self.otoi[o] for o in item if o in self.otoi]
    
    def deproc1(self, idx): return [self.vocab[i] for i in idx]

class MultiCategoryGetter(ItemGetter):
    _default_proc = MultiCategoryProcessor
    
    def get(self, o): return onehot(o, len(self.procs[0].vocab))
    def raw(self, o): return [i for i,x in enumerate(o) if x == 1]
    def show(self, x, ax): ax.set_title(';'.join(x))

In [ ]:
# export
def read_column(df, col_name, prefix='', suffix='', delim=None):
    values = df[col_name].values.astype(str)
    values = np.char.add(np.char.add(prefix, values), suffix)
    if delim is not None:
        values = np.array(list(csv.reader(values, delimiter=delim)))
    return values

In [ ]:
class PlanetData(DataBlock):
    get_x_cls = ImageGetter
    get_y_cls = MultiCategoryGetter
    
    def get_source(self):        
        self.path = untar_data(URLs.PLANET_SAMPLE)
        return pd.read_csv(path/'labels.csv')
    def get_items(self, source): return read_column(source, 'image_name', prefix=f'{self.path}/train/', suffix='.jpg')
    def split(self, items):      return random_splitter(items)
    def label(self, items):      return read_column(self.source, 'tags', delim=' ')

In [ ]:
data = PlanetData(tfms=tfms).databunch()

In [ ]:
data.show_batch()

### Camvid

In [ ]:
# export
class SegmentMaskGetter(ImageGetter):
    def __init__(self, procs=None, cmap='tab20', alpha=0.5): 
        super().__init__(procs, cmap=cmap, alpha=alpha)

In [ ]:
class CamvidData(DataBlock):
    get_x_cls = ImageGetter
    get_y_cls = SegmentMaskGetter
    
    def get_source(self):        return untar_data(URLs.CAMVID_TINY)      
    def get_items(self, source): return get_image_files(source/'images')
    def split(self, items):      return random_splitter(items)
    def label(self, items):      
        path_lbl = self.source/'labels'
        codes = np.loadtxt(self.source/'codes.txt', dtype=str)
        return func_labeller(items, lambda x: path_lbl/f'{x.stem}_P{x.suffix}')

In [ ]:
data = CamvidData(tfms=tfms, tfm_y=TfmY.Mask).databunch(bs=16)

In [ ]:
data.show_batch()

### Biwii

In [ ]:
import pickle

In [ ]:
# export
class PointsGetter(ItemGetter):
    def __init__(self, procs=None, do_scale=True, y_first=False): 
        super().__init__(procs)
        self.do_scale,self.y_first = do_scale,y_first
    
    def get(self, o):
        if not isinstance(o, torch.Tensor): o = tensor(o)
        o = o.view(-1, 2).float()
        if self.do_scale and hasattr(self, '_x') and self._x is not None: 
            sz = tensor([self._x.size[1], self._x.size[0]]).float()
            o = o * 2/sz - 1
        return o if self.y_first else o.flip(1)
    
    def raw(self, o):
        if hasattr(self, '_x') and self._x is not None: 
            sz = tensor(self._x.shape[1:]).float()
            o = (o + 1) * sz/2
        return o
    
    def show(self, x, ax):
        params = {'s': 10, 'marker': '.', 'c': 'r'}
        ax.scatter(x[:, 0], x[:, 1], **params)

In [ ]:
path = untar_data(URLs.BIWI_SAMPLE)
fnames = get_image_files(path/'images')
fn2ctr = pickle.load(open(path/'centers.pkl', 'rb'))

In [ ]:
fn2ctr[fnames[0].name]

In [ ]:
class ResizeFixed(Transform):
    _order=10
    def __init__(self, size, mode_x=PIL.Image.BILINEAR, mode_y=None):
        if isinstance(size,int): size=(size,size)
        size = (size[1],size[0]) #PIL takes size in the otherway round
        self.size,self.mode_x,self.mode_y = size,mode_x,mode_y
        
    def apply(self, x):       return x.resize(self.size, self.mode_x)
    def apply_image(self, y): return y.resize(self.size, self.mode_x if self.mode_y is None else self.mode_y)
    def apply_mask(self, y):  return y.resize(self.size, PIL.Image.NEAREST if self.mode_y is None else self.mode_y)
    def apply_point(self, y):
        rsz = tensor([self.x.size[1]/self.size[1], self.x.size[0]/self.size[0]], device=y.device)
        return (y+1) * rsz - 1
    def apply_bbox(self, y): return self.apply_point(y[0]),y[1]

In [ ]:
class BiwiData(DataBlock):
    get_x_cls = ImageGetter
    get_y_cls = PointsGetter
    
    def get_source(self):        return untar_data(URLs.BIWI_SAMPLE)      
    def get_items(self, source): return get_image_files(source/'images')
    def split(self, items):      return random_splitter(items)
    def label(self, items):      
        fn2ctr = pickle.load(open(self.source/'centers.pkl', 'rb'))
        return func_labeller(items, lambda o:fn2ctr[o.name])

In [ ]:
data = BiwiData(tfms=[DecodeImg(), ResizeFixed(128), ToByteTensor(), ToFloatTensor()], tfm_y=TfmY.Point).databunch(bs=16)

In [ ]:
data.show_batch()

In [ ]:
data.train_ds.x[0].size

In [ ]:
data.train_ds.y[0]

In [ ]:
%debug

## Export

In [ ]:
! python notebook2script.py "200_datablock_config.ipynb"